#### application 데이터 세트에 다양한 feature engineering 수행
* application_train(test) 주요 feature 값들의 분포도등의 EDA 수행
* application_train(test) 주요 feature 들의 추가적인 가공을 통한 feature engineering 수행

#### 라이브러리와 app 데이터 세트 로딩

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc, os, sys
import random
import seaborn as sns

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_colwidth', 30)

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
default_url = '/content/gdrive/MyDrive/home-credit-default-risk'
app_train = pd.read_csv(os.path.join(default_url, 'application_train.csv'))
app_test = pd.read_csv(os.path.join(default_url, 'application_test.csv'))

In [ ]:
app_train.isnull().sum()

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train.info()

#### 연속형 숫자 feature 들에 대해서 TARGET 값이 각각 0과 1일때의 Histogram 시각화

* violinplot과 distplot으로 숫자형 feature들에 대해 TARGET값 별 분포도 비교 시각화

In [ ]:
num_columns = app_train.dtypes[app_train.dtypes != 'object']
print(num_columns)

In [ ]:
def show_hist_by_target(df, columns):
    cond_1 = df['TARGET'] == 1
    cond_0 = df['TARGET'] == 0

    for column in columns:
        fig, axs = plt.subplots(figsize=(12,4), nrows=1, ncols=2, squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=axs[0][0])
        sns.distplot(df[cond_0][column], label='0', color='blue', ax=axs[0][1])
        sns.distplot(df[cond_1][column], label='1', color='red', ax=axs[0][1])

In [ ]:
columns = app_train.dtypes[app_train.dtypes == 'float64'].index.tolist()
show_hist_by_target(app_train, columns)

* AMT_INCOME_TOTAL, AMT_CREDIT 은 크게 차이가 없음. 
* AMT_ANNUITY는 TARGET=1시 상대적으로 작은 숫자대의 값이 조금 많음. 
* AMT_GOOD_PRICE는 크게 차이가 없음. 
* DAYS_BIRTH는 TARGET=1시 적은 연령대의 숫자값이 상대적으로 많음. 
* DAYS_EMPLOYED는 TARGET=1시 작은 값들이 조금 더 많음. 
* DAYS_ID_PUBLISH, DAYS_REGISTRATION  는 TARGET=1시 최근 값들이 조금 더 많음. 
* DAYS_LAST_PHONE_CHANGE는 큰 차이 없음. 
* CNT_FAM_MEMBERS는 차이가 없음. outlier 때문에 histogram에 차이가 있어 보임. 
* REGION_RATING_CLIENT는 큰 차이 없음 
* EXT_SOURCE_1,EXT_SOURCE_2, EXT_SOURCE_3 모두 조금씩 차이가 있음.  
* 나머지 컬럼모두 큰 차이가 없음. 
* 전반적으로 연령대가 낮은(직장 경력이 적은), 소액 대출에서 상대적으로 연체 비중이 높음.  

#### seaborn의 countplot() 또는 catplot()을 이용하여 category 피처(object 컬럼)을 TARGET 유형에 따라 Count 비교

In [ ]:
object_columns = app_train.dtypes[app_train.dtypes == 'object'].index.tolist()

In [ ]:
def show_count_by_target(df, columns):
    cond_1 = df['TARGET'] == 1
    cond_0 = df['TARGET'] == 0

    for column in columns:
        fig, axs = plt.subplots(figsize= (12,4), nrows=1, ncols=2, squeeze=False)
        # countplot을 이용하여 category값의 histogram 표현
        chart0 = sns.countplot(df[cond_0][column], ax=axs[0][0])
        # x축의 tick label들이 값 유형이 많으므로 45도로 회전하여 표현
        chart0.set_xticklabels(chart0.get_xticklabels(), rotation=45)
        chart1 = sns.countplot(df[cond_1][column], ax=axs[0][1])
        chart1.set_xticklabels(chart1.get_xticklabels(),rotation=45)

show_count_by_target(app_train, object_columns)

In [ ]:
# catplot을 이용하여 countplot을 특정 컬럼 값 조건에 따라 여러개의 subplot을 분리하여 보여줌
sns.catplot(x='CODE_GENDER', col='TARGET', data=app_train, kind='count')

In [ ]:
def show_category_by_target(df, columns):
    for column in columns:
        chart = sns.catplot(x=column, col='TARGET', data=df, kind='count')
        chart.set_xticklabels(rotation=45)
show_category_by_target(app_train, object_columns)

#### 대출 횟수 대비 연체 비율이 여성보다 남성이 높음 이를 value_counts()로 확인

In [ ]:
cond_0 = app_train['TARGET'] == 0
cond_1 = app_train['TARGET'] == 1
cond_m = app_train['CODE_GENDER'] == 'M'
cond_f = app_train['CODE_GENDER'] == 'F'

# 전체 건수 대비 남성과 여성의 비율 확인
print(app_train['CODE_GENDER'].value_counts()/app_train.shape[0])
# TARGET = 1일 경우 남성과 여성의 비율 확인
print(app_train[cond_1]['CODE_GENDER'].value_counts()/app_train[cond_1].shape[0])
# TARGET = 0일 경우 남성과 여성의 비율 확인
print(app_train[cond_0]['CODE_GENDER'].value_counts()/app_train[cond_0].shape[0])

In [ ]:
corr_columns = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
               'DAYS_EMPLOYED','DAYS_ID_PUBLISH', 'DAYS_REGISTRATION', 'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL', 'TARGET']

corr = app_train[corr_columns].corr()

#### 주요 컬럼들의 target과의 상관도 분석

In [ ]:
plt.figure(figsize=(9,9))
sns.heatmap(corr, annot=True)

#### 이상치 데이터 확인 및 DAYS_EMPLOYED 이상치 값 변경

In [ ]:
# 365243이 매우 많음 약 1000년치에 해당하는 날짜
app_train['DAYS_EMPLOYED'].value_counts()

In [ ]:
# CODE_GENDER의 경우 XNA가 4건 정도 많지 않으므로 그대로 유지
app_train['CODE_GENDER'].value_counts()

In [ ]:
# LightGBM은 NULL값을 트리 모델 생성하는데 사용할 수 있으므로 일괄적으로 Null로 변환

app_train['DAYS_EMPLOYED'] = app_train['DAYS_EMPLOYED'].replace(365243, np.nan)
app_train['DAYS_EMPLOYED'].value_counts(dropna=False)

#### 주요 Feature들에 대한 feature engineering 수행

#### EXT_SOURCE 계열값 확인, EXT_SOURCE_X 피처들의 평균/최대/최소/표준편차 확인

In [ ]:
app_train[['EXT_SOURCE_1','EXT_SOURCE_2', 'EXT_SOURCE_3']].isnull().sum()

In [ ]:
app_train['EXT_SOURCE_1'].value_counts(dropna=False)

In [ ]:
app_train['EXT_SOURCE_2'].value_counts(dropna=False)

In [ ]:
app_train['EXT_SOURCE_3'].value_counts(dropna=False)

In [ ]:
# EXT_SOURCE_X 피처들의 평균/최대/최소/표준편차 확인

print('### mean ###\n', app_train[['EXT_SOURCE_1','EXT_SOURCE_2', 'EXT_SOURCE_3']].mean())
print('### max ###\n', app_train[['EXT_SOURCE_1','EXT_SOURCE_2', 'EXT_SOURCE_3']].max())
print('### min ###\n', app_train[['EXT_SOURCE_1','EXT_SOURCE_2', 'EXT_SOURCE_3']].min())
print('### std ###\n', app_train[['EXT_SOURCE_1','EXT_SOURCE_2', 'EXT_SOURCE_3']].std())

In [ ]:
apps = pd.concat([app_train, app_test])
print(apps.shape)

#### EXT_SOURCE_X FEATURE 가공

* EXT_SOURCE_X 피처들을 결합하여 평균과 표준편차를 신규 생성

In [ ]:
# EXT_SOURCE_X 피처 들을 결합하여 평균과 표준편차를 신규 생성
apps['APP_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].mean(axis=1)
apps['APP_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].std(axis=1)

In [ ]:
apps['APP_EXT_SOURCE_STD'].isnull().sum()

In [ ]:
# 새로 생성한 APPS_EXT_SOURCE_STD이 NULL일 경우 APPS_EXT_SOURCE_STD의 평균으로 일괄 변경. 
apps['APP_EXT_SOURCE_STD'] = apps['APP_EXT_SOURCE_STD'].fillna(apps['APP_EXT_SOURCE_STD'].mean())
apps['APP_EXT_SOURCE_STD'].isnull().sum()

#### AMT_CREDIT 비율로 Feature 가공

In [ ]:
# AMT_CREDIT(대출 금액)으로 대출과 관련된 금액 추가 가공.
apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY'] / apps['AMT_CREDIT']
apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE'] / apps['AMT_CREDIT']
apps['APPS_CREDIT_GOODS_DIFF'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']

AMT_INCOME_TOTAL 비율로 Feature 가공

In [ ]:
# AMT_INCOME_TOTAL 비율로 대출 금액 관련 피처 가공
apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY'] / apps['AMT_INCOME_TOTAL']
apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT'] / apps['AMT_INCOME_TOTAL']
apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE'] / apps['AMT_INCOME_TOTAL']
# 가족수를 고려한 가처분 소득 피처 가공. 
apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL'] / apps['CNT_FAM_MEMBERS']

#### DAYS_BIRTH, DAYS_EMPLOYED 비율로 Feature 가공.

In [ ]:
# DAYS_BIRTH, DAYS_EMPLOYED 비율로 소득/자산 관련 Feature 가공. 
apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED'] / apps['DAYS_BIRTH']
apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL'] / apps['DAYS_EMPLOYED']
apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL'] / apps['DAYS_BIRTH']
apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']

#### 데이터 레이블 인코딩, NULL값은 LightGBM 내부에서 처리하도록 특별한 변경하지 않음. 

In [ ]:
object_columns = apps.dtypes[apps.dtypes=='object'].index.tolist()
for column in object_columns:
    apps[column] = pd.factorize(apps[column])[0]

#### 학습 데이터와 테스트 데이터 다시 분리

In [ ]:
app_train = apps[~apps['TARGET'].isnull()]
app_test = apps[apps['TARGET'].isnull()]

app_test = app_test.drop('TARGET', axis=1)

#### 학습 데이터를 검증 데이터로 분리하고 LGBM Classifier로 학습 수행. 

In [ ]:
from sklearn.model_selection import train_test_split

ftr_app = app_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = app_train['TARGET']

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size= 0.3, random_state =2020)
print(train_x.shape, valid_x.shape)

#### 학습된 Classifier를 이용하여 테스트 데이터 예측하고 결과를 Kaggle로 Submit 수행. 

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
    n_jobs=-1,
    n_estimators=1000,
    learning_rate=0.02,
    num_leaves=32,
    subsample=0.8,
    max_depth=12,
    silent=-1,
    verbose=-1
)

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],
        eval_metric='auc', verbose=100, early_stopping_rounds=100)

##### 코랩 버전은 Google Drive로 예측 결과 CSV를 생성.

In [ ]:
preds = clf.predict_proba(app_test.drop(['SK_ID_CURR'], axis=1))[:,1]
app_test['TARGET'] = preds

In [ ]:
import os, sys 
from google.colab import drive 

drive.mount('/content/gdrive')

# SK_ID_CURR과 TARGET 값만 csv 형태로 생성. 코랩 버전은 구글 드라이브 절대 경로로 입력
app_test[['SK_ID_CURR', 'TARGET']].to_csv(os.path.join(default_url,'apps_baseline_02_01.csv'), index=False)

#### application 데이터 세트 feature engineering을 함수화

In [ ]:
def get_apps_processed(apps):
    
    # EXT_SOURCE_X FEATURE 가공
    apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())
      
    # AMT_CREDIT 비율로 Feature 가공
    apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
    apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
    
    # AMT_INCOME_TOTAL 비율로 Feature 가공
    apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
    apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']
    
    # DAYS_BIRTH, DAYS_EMPLOYED 비율로 Feature 가공
    apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
    apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
    apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
    apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
    apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']
       
    return apps

    